# Advanced OpenMC Tallies

All of these examples will use the the PWR assembly model provided by the `openmc.examples` module. We will use a power level for the assembly of 86.7 $\frac{kW}{cm}$ (this is a 2D model). Using this model, perform the following tasks:

1. Find the cell filled with the fuel material (uranium oxide) used in the lattice on the model. Hint: this cell is filled with a material named "Fuel".
2. Tally the power produced by each fuel_cell in the assembly and plot them using the `plot_power` function provided below.
3. Change the lower x and y planes of the model from a reflecting boundary condition to a vacuum boundary condition.
4. Use a `MeshFilter` with 85, 85, and 1 element in x, y, and z dimensions respectively to plot the new flux distribution of the assembly with `plt.imshow`.
5. Create a tally using the `MeshFilter` from the previous task to tally the flux. Include a `MaterialFilter` for all materials in the problem. Plot the power for the fuel material only using `plt.imshow`.
6. Use a `ZernikeFilter` of order 10 over the pincell at the center of the assembly (centered on the origin). You'll need to determine the radius of this fuel cell by examining it's `region` attribute. Apply this filter to a tally for the power. Increase the number of particles per batch to 1M particles and re-run the simulation.
7. Plot the angular distribution of the flux for the center pin using the `plot_radial` function provided below. What do you observe about the power distribution?

Some methods of the `openmc.Geometry` class on the `model` below that will be useful in this exercise:

  - `Geometry.get_all_lattices`: returns a dict of all lattices w/ their IDs as keys
  - `Geometry.get_all_material_cells`: returns a dict of all cells filled with materials w/ their IDs as keys

Other tips:

  - the `Region.get_surfaces` method will retrieve surfaces of a cell's `region` attribute. (i.e. `cell.region.get_surfaces`)
  - the attributes of the `Tally` class (`filters`, `scores`, `nuclides`) either inherit from the `list` type or _are_ a `list` and as a result support `list` operations (i.e. `append`, `pop`, indexing, etc.)

In [ ]:
import openmc
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
model = openmc.examples.pwr_assembly()
model.settings.particles = 100_000
model.settings.batches = 50
model.settings.inactive = 10
model.geometry.plot(pixels=(600, 600))
plt.show()

In [ ]:
import openmc.lib

def plot_power(model, cell, power_data, pixels=(600, 600)):
    """

    Parameters
    ----------

    model : openmc.Model
        Model used to determine pixel values for the plot
    power_tally : openmc.Tally
        Tally object containing power data (ordered by cell instances)

    Returns
    -------
    id_mapping : numpy.ndarray
        Mapping of cell IDs and instances to pixel values
    """

    try:
        openmc.lib.init()
        bbox = openmc.lib.global_bounding_box()
        p = openmc.lib.plot._PlotBase()
        p.origin = (0.0, 0.0, 0.0)
        p.width = bbox[1][0] - bbox[0][0]
        p.height = bbox[1][1] - bbox[0][1]
        p.h_res = pixels[0]
        p.v_res = pixels[1]
        p.color_by = 'cell'
        p.basis = 'xy'

        id_mapping = openmc.lib.id_map(p)
    finally:
        openmc.lib.finalize()

    # index into mapping for cell ID and instance

    image = np.nan * np.ones((pixels[0], pixels[1]))
    heating_mask = id_mapping[..., 0] == cell.id
    # populate image with instance values for the cell
    image[heating_mask] = id_mapping[heating_mask, 1]

    # replace instance values with power values
    for i, p in enumerate(power_data):
        image[image == i] = p

    plt.imshow(image, origin='lower', cmap='jet', extent=(bbox[0][0], bbox[1][0], bbox[0][1], bbox[1][1]))
    plt.colorbar(label='Power (W/cm)')
    plt.xlabel('X [cm]')
    plt.ylabel('Y [cm]')
    plt.show()

In [ ]:
def plot_radial(z_n, radius):
    """

    Parameters
    ----------

    z_n : Iterable of float
        The Zernike radial polynomial coefficients.
    radius : float
        The radius of the polynomial domain.

    """
    zz = openmc.Zernike(z_n, radius=radius)
    # Using linspace so that the endpoint of 360 is included...
    azimuths = np.radians(np.linspace(0, 360, 50))
    zeniths = np.linspace(0, radius, 100)
    r, theta = np.meshgrid(zeniths, azimuths)
    values = zz(zeniths, azimuths)
    fig, ax = plt.subplots(subplot_kw=dict(projection='polar'))
    ax.contourf(theta, r, values, cmap='jet')
    plt.show()

### Part 1: Find the cell containing the fuel material.

In [ ]:
fuel_material = None
for material in model.materials:
    "YOUR CODE HERE"
print(fuel_material)

In [ ]:
fuel_cell = None
for cell in model.geometry.get_all_material_cells().values():
    if cell.fill == "YOUR CODE HERE":
        fuel_cell = cell
print(fuel_cell)

### Part 2: Create a power tally for the fuel cell, determine the power produced by each instance of the fuel cell, and plot the power using the `plot_power` function.

In [ ]:

pin_heating_tally = openmc.Tally()
distribcell_filter = "YOUR CODE HERE"
pin_heating_tally.filters = [distribcell_filter]
pin_heating_tally.scores = ['heating']

total_heating_tally = openmc.Tally()
total_heating_tally.scores = "YOUR CODE HERE"

model.tallies = [pin_heating_tally, total_heating_tally]

sp_file = model.run()

In [ ]:
with openmc.StatePoint(sp_file) as sp:
    pin_heating_tally_results = sp.tallies[pin_heating_tally.id]
    total_heating_tally = sp.tallies[total_heating_tally.id]

"YOUR CODE HERE"
print(pin_powers)

In [ ]:
plot_power(model, fuel_cell, pin_powers, pixels=(600, 600))

In [ ]:
model.geometry.root_universe.cells

In [ ]:
root_cell = list(model.geometry.root_universe.cells.values())[0]
surfaces = "YOUR CODE HERE"
print(surfaces)

In [ ]:
# set two of the surface boundary conditions to vacuum
for surface in surfaces.values():
    "YOUR CODE HERE"

In [ ]:
print(surfaces)

### Part 4: Use a MeshFilter to get the flux distribution

In [ ]:
"YOUR CODE HERE" # determine bounds of the model

mesh = openmc.RegularMesh()
mesh.lower_left = "YOUR CODE HERE"
mesh.upper_right = "YOUR CODE HERE"
mesh.dimension = "YOUR CODE HERE"

mesh_filter = "YOUR CODE HERE"
mesh_tally = openmc.Tally()
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['flux']

model.tallies = [mesh_tally]

sp_file = model.run()


In [ ]:
with openmc.StatePoint(sp_file) as sp:
    mesh_tally_results = sp.tallies[mesh_tally.id]

# isolate the mesh data and reshape it for visualization appropriately
"YOUR CODE HERE"
flux = "YOUR CODE HERE"
plt.imshow(flux, origin='lower')
plt.colorbar(label='Flux')
plt.xlabel('X [cm]')
plt.ylabel('Y [cm]')
plt.show()


### Part 5: Add a material filter and plot power of fuel material

In [ ]:
materials = model.materials
"YOUR CODE HERE"
sp_file = model.run()


In [ ]:
with openmc.StatePoint(sp_file) as sp:
    mesh_tally_results = sp.tallies[mesh_tally.id]

# get results only the  fuel material
flux_fuel = mesh_tally_results.get_values("YOUR CODE HERE")
# reshape for visualization
flux_fuel_image = "YOUR CODE HERE"
plt.imshow(flux_fuel_image, origin='lower')
plt.colorbar(label='Flux')
plt.xlabel('X [cm]')
plt.ylabel('Y [cm]')
plt.show()

### Part 6: Apply a ZernikeRadialFilter to the center pin domain

In [ ]:
# display fuel cell surfaces
"YOUR CODE HERE"

In [ ]:
order = "YOUR CODE HERE"
radius = "YOUR CODE HERE" # cm
cell_filter = openmc.CellFilter([fuel_cell])

# Create a Zernike azimuthal polynomial expansion filter and add to tally
flux_tally_zernike = openmc.Tally()
flux_tally_zernike.scores = ['flux']
zernike_filter = openmc.ZernikeFilter("YOUR CODE HERE")
flux_tally_zernike.filters = [zernike_filter]

model.tallies = [flux_tally_zernike]
model.settings.particles = 1_000_000

sp_file = model.run()

### Part 7: Plot the result of the tally with the ZernikeFilter applied.

In [ ]:
with openmc.StatePoint(sp_file) as sp:
    print(sp.tallies)
    flux_tally_zernike_results = sp.tallies[flux_tally_zernike.id]


In [ ]:
# extract the Zernike polynomial coefficients and plot
z_n = "YOUR CODE HERE"
plot_radial(z_n, radius)